# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

1. Create a new DataFrame that only includes customers who have a total_claim_amount greater than $1,000 and have a response of "Yes" to the last marketing campaign.

In [3]:
import pandas as pd

# Define the URL of the CSV file
url = "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv"

# Download the CSV file
df = pd.read_csv(url)

# Display the first few rows of the dataframe to confirm successful download
print(marketing_customer_analysis.head())


   Unnamed: 0 Customer       State  Customer Lifetime Value Response  \
0           0  DK49336     Arizona              4809.216960       No   
1           1  KX64629  California              2228.525238       No   
2           2  LZ68649  Washington             14947.917300       No   
3           3  XL78013      Oregon             22332.439460      Yes   
4           4  QA50777      Oregon              9025.067525       No   

   Coverage Education Effective To Date EmploymentStatus Gender  ...  \
0     Basic   College           2/18/11         Employed      M  ...   
1     Basic   College           1/18/11       Unemployed      F  ...   
2     Basic  Bachelor           2/10/11         Employed      M  ...   
3  Extended   College           1/11/11         Employed      M  ...   
4   Premium  Bachelor           1/17/11    Medical Leave      F  ...   

   Number of Open Complaints Number of Policies     Policy Type        Policy  \
0                        0.0                  9  Corp

In [4]:
# Filter the DataFrame for customers with total_claim_amount > 1000 and Response = "Yes"
filtered_df = df[(df['Total Claim Amount'] > 1000) & (df['Response'] == "Yes")]

# Display the first few rows of the filtered DataFrame
print(filtered_df.head())


     Unnamed: 0 Customer       State  Customer Lifetime Value Response  \
189         189  OK31456  California             11009.130490      Yes   
236         236  YJ16163      Oregon             11009.130490      Yes   
419         419  GW43195      Oregon             25807.063000      Yes   
442         442  IP94270     Arizona             13736.132500      Yes   
587         587  FJ28407  California              5619.689084      Yes   

     Coverage             Education Effective To Date EmploymentStatus Gender  \
189   Premium              Bachelor           1/24/11         Employed      F   
236   Premium              Bachelor           1/24/11         Employed      F   
419  Extended               College           2/13/11         Employed      F   
442   Premium                Master           2/13/11         Disabled      F   
587   Premium  High School or Below           1/26/11       Unemployed      M   

     ...  Number of Open Complaints Number of Policies     Policy Ty

2. Using the original Dataframe, analyze the average total_claim_amount by each policy type and gender for customers who have responded "Yes" to the last marketing campaign. Write your conclusions.

In [5]:
# Calculate the average Total Claim Amount by Policy Type and Gender for customers who responded "Yes"
average_claim_by_policy_gender = df[df['Response'] == "Yes"].groupby(['Policy Type', 'Gender'])['Total Claim Amount'].mean().reset_index()

# Display the results
print(average_claim_by_policy_gender)


      Policy Type Gender  Total Claim Amount
0  Corporate Auto      F          433.738499
1  Corporate Auto      M          408.582459
2   Personal Auto      F          452.965929
3   Personal Auto      M          457.010178
4    Special Auto      F          453.280164
5    Special Auto      M          429.527942


3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

In [6]:
# Count the number of customers in each state
customer_count_by_state = df['State'].value_counts().reset_index()
customer_count_by_state.columns = ['State', 'Customer Count']

# Filter the states where there are more than 500 customers
filtered_states = customer_count_by_state[customer_count_by_state['Customer Count'] > 500]

# Display the filtered results
print(filtered_states)


        State  Customer Count
0  California            3552
1      Oregon            2909
2     Arizona            1937
3      Nevada             993
4  Washington             888


4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

In [7]:
# Group by Education and Gender, then calculate max, min, and median for Customer Lifetime Value
clv_stats = df.groupby(['Education', 'Gender'])['Customer Lifetime Value'].agg(['max', 'min', 'median']).reset_index()

# Display the results
print(clv_stats)


              Education Gender          max          min       median
0              Bachelor      F  73225.95652  1904.000852  5640.505303
1              Bachelor      M  67907.27050  1898.007675  5548.031892
2               College      F  61850.18803  1898.683686  5623.611187
3               College      M  61134.68307  1918.119700  6005.847375
4                Doctor      F  44856.11397  2395.570000  5332.462694
5                Doctor      M  32677.34284  2267.604038  5577.669457
6  High School or Below      F  55277.44589  2144.921535  6039.553187
7  High School or Below      M  83325.38119  1940.981221  6286.731006
8                Master      F  51016.06704  2417.777032  5729.855012
9                Master      M  50568.25912  2272.307310  5579.099207


## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

In [8]:
import pandas as pd

# Ensure the Effective To Date column is in datetime format
df['Effective To Date'] = pd.to_datetime(df['Effective To Date'])

# Extract the month from the Effective To Date column
df['Month'] = df['Effective To Date'].dt.month

# Create a pivot table with states as rows, months as columns, and the count of policies as values
policies_by_state_month = df.pivot_table(index='State', columns='Month', aggfunc='size', fill_value=0)

# Rename the columns to represent the actual month names (optional)
policies_by_state_month.columns = pd.to_datetime(policies_by_state_month.columns, format='%m').strftime('%B')

# Display the pivot table
print(policies_by_state_month)


Month       January  February
State                        
Arizona        1008       929
California     1918      1634
Nevada          551       442
Oregon         1565      1344
Washington      463       425


7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9

In [9]:
import pandas as pd

# Calculate the total number of customers by each marketing channel
total_customers_by_channel = df['Sales Channel'].value_counts().reset_index()
total_customers_by_channel.columns = ['Sales Channel', 'Total Customers']

# Calculate the number of customers who responded "Yes" by each marketing channel
yes_responses_by_channel = df[df['Response'] == 'Yes']['Sales Channel'].value_counts().reset_index()
yes_responses_by_channel.columns = ['Sales Channel', 'Yes Responses']

# Merge the two DataFrames on Sales Channel
response_rate_df = pd.merge(yes_responses_by_channel, total_customers_by_channel, on='Sales Channel')

# Calculate the response rate
response_rate_df['Response Rate'] = (response_rate_df['Yes Responses'] / response_rate_df['Total Customers']) * 100

# Sort by Response Rate in descending order
response_rate_df = response_rate_df.sort_values(by='Response Rate', ascending=False)

# Display the response rate DataFrame
print(response_rate_df)


  Sales Channel  Yes Responses  Total Customers  Response Rate
0         Agent            742             4121      18.005339
3           Web            177             1626      10.885609
1        Branch            326             3022      10.787558
2   Call Center            221             2141      10.322279
